<a href="https://colab.research.google.com/github/ninadcpa/DSC540/blob/main/Week_7_8_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DSC540 Week 7 and 8 Exercise 
@author Ninad Patkhedkar

In [1]:
# This code is written for mounting code folder checked out from github on google drive
# All assignment code will be maintained inside folder "DSC540"
# Code from book is inside folder "Data-Wrangling-with-Python"

from google.colab import drive
import sys
import os

drive.mount('/content/drive')
sys.path.append('/content/drive/MyDrive/Colab Notebooks/DSC540')
os.chdir('/content/drive/MyDrive/Colab Notebooks/DSC540')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


The four chapters you read these two weeks focus extensively on cleaning and transforming data.

I am choosing below dataset for this exercise

*  [So Much Data Candy, Seriously (Ng, 2017)](https://www.scq.ubc.ca/wp-content/uploads/2017/10/candyhierarchy2017.csv) 

I have downloaded the data and will use local file.


In [2]:
import numpy as np
import pandas as pd

# File was throwing some encoding error while trying to read with default utf-8 encoding
# Using latin-1 encoding to fix issue
# Missing values or spaces would be replaced by np.nan
na_values = [""," ",None,"Missing","NA"]
df=pd.read_csv("candyhierarchy2017.csv",encoding='latin-1',na_values=na_values)
df.head()

,Internal ID,Q1: GOING OUT?,Q2: GENDER,Q3: AGE,Q4: COUNTRY,"Q5: STATE, PROVINCE, COUNTY, ETC",Q6 | 100 Grand Bar,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),Q6 | Any full-sized candy bar,Q6 | Black Jacks,Q6 | Bonkers (the candy),Q6 | Bonkers (the board game),Q6 | Bottle Caps,Q6 | Box'o'Raisins,Q6 | Broken glow stick,Q6 | Butterfinger,Q6 | Cadbury Creme Eggs,Q6 | Candy Corn,Q6 | Candy that is clearly just the stuff given out for free at restaurants,Q6 | Caramellos,"Q6 | Cash, or other forms of legal tender",Q6 | Chardonnay,Q6 | Chick-o-Sticks (we donÕt know what that is),Q6 | Chiclets,Q6 | Coffee Crisp,Q6 | Creepy Religious comics/Chick Tracts,Q6 | Dental paraphenalia,Q6 | Dots,Q6 | Dove Bars,Q6 | Fuzzy Peaches,Q6 | Generic Brand Acetaminophen,Q6 | Glow sticks,Q6 | Goo Goo Clusters,Q6 | Good N' Plenty,Q6 | Gum from baseball cards,Q6 | Gummy Bears straight up,Q6 | Hard Candy,Q6 | Healthy Fruit,Q6 | Heath Bar,Q6 | Hershey's Dark Chocolate,...,Q6 | Real Housewives of Orange County Season 9 Blue-Ray,Q6 | ReeseÕs Peanut Butter Cups,Q6 | Reese's Pieces,Q6 | Reggie Jackson Bar,Q6 | Rolos,Q6 | Sandwich-sized bags filled with BooBerry Crunch,Q6 | Skittles,Q6 | Smarties (American),Q6 | Smarties (Commonwealth),Q6 | Snickers,Q6 | Sourpatch Kids (i.e. abominations of nature),Q6 | Spotted Dick,Q6 | Starburst,Q6 | Sweet Tarts,Q6 | Swedish Fish,Q6 | Sweetums (a friend to diabetes),Q6 | Take 5,Q6 | Tic Tacs,Q6 | Those odd marshmallow circus peanut things,Q6 | Three Musketeers,Q6 | Tolberone something or other,Q6 | Trail Mix,Q6 | Twix,"Q6 | Vials of pure high fructose corn syrup, for main-lining into your vein",Q6 | Vicodin,Q6 | Whatchamacallit Bars,Q6 | White Bread,Q6 | Whole Wheat anything,Q6 | York Peppermint Patties,Q7: JOY OTHER,Q8: DESPAIR OTHER,Q9: OTHER COMMENTS,Q10: DRESS,Unnamed: 113,Q11: DAY,Q12: MEDIA [Daily Dish],Q12: MEDIA [Science],Q12: MEDIA [ESPN],Q12: MEDIA [Yahoo],"Click Coordinates (x, y)"
0,90258773,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,90272821,No,Male,44,USA,NM,MEH,DESPAIR,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,MEH,DESPAIR,MEH,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,DESPAIR,MEH,MEH,DESPAIR,MEH,JOY,...,DESPAIR,JOY,JOY,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,JOY,JOY,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,Mounds,NaN,Bottom line is Twix is really the only candy w...,White and gold,NaN,Sunday,NaN,1.0,NaN,NaN,"(84, 25)"
2,90272829,NaN,Male,49,USA,Virginia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,90272840,No,Male,40,us,or,MEH,DESPAIR,JOY,MEH,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,MEH,DESPAIR,DESPAIR,MEH,JOY,MEH,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,JOY,JOY,MEH,MEH,JOY,JOY,MEH,DESPAIR,JOY,MEH,MEH,DESPAIR,MEH,...,DESPAIR,JOY,MEH,MEH,MEH,DESPAIR,DESPAIR,DESPAIR,MEH,JOY,MEH,DESPAIR,MEH,MEH,JOY,MEH,JOY,MEH,DESPAIR,DESPAIR,JOY,MEH,JOY,DESPAIR,JOY,JOY,DESPAIR,DESPAIR,DESPAIR,"Reese's crispy crunchy bars, 5th avenue bars, ...",NaN,Raisins can go to hell,White and gold,NaN,Sunday,NaN,1.0,NaN,NaN,"(75, 23)"
4,90272841,No,Male,23,usa,exton pa,JOY,DESPAIR,JOY,DESPAIR,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,MEH,DESPAIR,DESPAIR,JOY,MEH,JOY,DESPAIR,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,JOY,DESPAIR,DESPAIR,DESPAIR,MEH,MEH,DESPAIR,JOY,MEH,DESPAIR,DESPAIR,JOY,...,DESPAIR,JOY,JOY,MEH,JOY,NaN,JOY,JOY,DESPAIR,NaN,MEH,DESPAIR,JOY,

Perform 2 tasks out of below list

Chapter 7
* Filter out missing data
* Fill in missing data
* Remove duplicates
* Transform data using either mapping or a function
* Replace values
* Discretization and Binning
* Manipulate Strings
* Noticed that missing Values are automatically populated as NaN



In [3]:
df.shape

(2460, 120)

In [4]:
df.isna().sum()

Internal ID                    0
Q1: GOING OUT?               110
Q2: GENDER                    41
Q3: AGE                       84
Q4: COUNTRY                   64
                            ... 
Q12: MEDIA [Daily Dish]     2375
Q12: MEDIA [Science]        1098
Q12: MEDIA [ESPN]           2361
Q12: MEDIA [Yahoo]          2393
Click Coordinates (x, y)     855
Length: 120, dtype: int64

Internal ID value is unique. I will use Interanl ID value as index.


In [5]:
df.set_index(['Internal ID'],inplace=True)
df.head()

,Q1: GOING OUT?,Q2: GENDER,Q3: AGE,Q4: COUNTRY,"Q5: STATE, PROVINCE, COUNTY, ETC",Q6 | 100 Grand Bar,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),Q6 | Any full-sized candy bar,Q6 | Black Jacks,Q6 | Bonkers (the candy),Q6 | Bonkers (the board game),Q6 | Bottle Caps,Q6 | Box'o'Raisins,Q6 | Broken glow stick,Q6 | Butterfinger,Q6 | Cadbury Creme Eggs,Q6 | Candy Corn,Q6 | Candy that is clearly just the stuff given out for free at restaurants,Q6 | Caramellos,"Q6 | Cash, or other forms of legal tender",Q6 | Chardonnay,Q6 | Chick-o-Sticks (we donÕt know what that is),Q6 | Chiclets,Q6 | Coffee Crisp,Q6 | Creepy Religious comics/Chick Tracts,Q6 | Dental paraphenalia,Q6 | Dots,Q6 | Dove Bars,Q6 | Fuzzy Peaches,Q6 | Generic Brand Acetaminophen,Q6 | Glow sticks,Q6 | Goo Goo Clusters,Q6 | Good N' Plenty,Q6 | Gum from baseball cards,Q6 | Gummy Bears straight up,Q6 | Hard Candy,Q6 | Healthy Fruit,Q6 | Heath Bar,Q6 | Hershey's Dark Chocolate,Q6 | HersheyÕs Milk Chocolate,...,Q6 | Real Housewives of Orange County Season 9 Blue-Ray,Q6 | ReeseÕs Peanut Butter Cups,Q6 | Reese's Pieces,Q6 | Reggie Jackson Bar,Q6 | Rolos,Q6 | Sandwich-sized bags filled with BooBerry Crunch,Q6 | Skittles,Q6 | Smarties (American),Q6 | Smarties (Commonwealth),Q6 | Snickers,Q6 | Sourpatch Kids (i.e. abominations of nature),Q6 | Spotted Dick,Q6 | Starburst,Q6 | Sweet Tarts,Q6 | Swedish Fish,Q6 | Sweetums (a friend to diabetes),Q6 | Take 5,Q6 | Tic Tacs,Q6 | Those odd marshmallow circus peanut things,Q6 | Three Musketeers,Q6 | Tolberone something or other,Q6 | Trail Mix,Q6 | Twix,"Q6 | Vials of pure high fructose corn syrup, for main-lining into your vein",Q6 | Vicodin,Q6 | Whatchamacallit Bars,Q6 | White Bread,Q6 | Whole Wheat anything,Q6 | York Peppermint Patties,Q7: JOY OTHER,Q8: DESPAIR OTHER,Q9: OTHER COMMENTS,Q10: DRESS,Unnamed: 113,Q11: DAY,Q12: MEDIA [Daily Dish],Q12: MEDIA [Science],Q12: MEDIA [ESPN],Q12: MEDIA [Yahoo],"Click Coordinates (x, y)"
Internal ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
90258773,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90272821,No,Male,44,USA,NM,MEH,DESPAIR,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,MEH,DESPAIR,MEH,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,DESPAIR,MEH,MEH,DESPAIR,MEH,JOY,JOY,...,DESPAIR,JOY,JOY,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,JOY,JOY,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,Mounds,NaN,Bottom line is Twix is really the only candy w...,White and gold,NaN,Sunday,NaN,1.0,NaN,NaN,"(84, 25)"
90272829,NaN,Male,49,USA,Virginia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90272840,No,Male,40,us,or,MEH,DESPAIR,JOY,MEH,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,MEH,DESPAIR,DESPAIR,MEH,JOY,MEH,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,JOY,JOY,MEH,MEH,JOY,JOY,MEH,DESPAIR,JOY,MEH,MEH,DESPAIR,MEH,MEH,...,DESPAIR,JOY,MEH,MEH,MEH,DESPAIR,DESPAIR,DESPAIR,MEH,JOY,MEH,DESPAIR,MEH,MEH,JOY,MEH,JOY,MEH,DESPAIR,DESPAIR,JOY,MEH,JOY,DESPAIR,JOY,JOY,DESPAIR,DESPAIR,DESPAIR,"Reese's crispy crunchy bars, 5th avenue bars, ...",NaN,Raisins can go to hell,White and gold,NaN,Sunday,NaN,1.0,NaN,NaN,"(75, 23)"
90272841,No,Male,23,usa,exton pa,JOY,DESPAIR,JOY,DESPAIR,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,MEH,DESPAIR,DESPAIR,JOY,MEH,JOY,DESPAIR,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,JOY,DESPAIR,DESPAIR,DES

There are lot of missing values.
We will delete all rows for which all values are missing i.e. NaN

In [6]:
df.dropna(axis=0,how='all',inplace=True)
df.shape

(2439, 119)

It shows that around 21 rows dropped as it didn't had any value populated.

Check if there are any duplicate records.

In [7]:
df.duplicated().sum()

15

Drop duplicates.

In [8]:
df.drop_duplicates(inplace=True)
df.shape

(2424, 119)

There are few columns for which more than 50% records have missing values i.e. NaN.

I don't think those columns would be of any use, hence dropping those columns.

In [9]:
# Dropping columns where more than 50% records are NaN
df.dropna(thresh=df.shape[0]*0.5,how='all',axis=1,inplace=True)
df.shape

(2424, 112)

In [10]:
# Dropping rows where more than 50% records are NaN
df.dropna(thresh=df.shape[1]*0.5,how='all',axis=0,inplace=True)
df.shape

(1781, 112)

Notice that 7 columns are dropped.

Country names are missing at some places and also in different cases like US, us and some places usa

In [11]:
df['Q4: COUNTRY'].unique()

array(['USA ', 'us', 'usa', nan, 'canada', 'Canada', 'Us', 'US', 'Murica',
       'USA', 'uk', 'United States', 'united states', 'Usa',
       'United staes', 'United Kingdom', 'United States of America',
       'UAE', 'United States ', 'canada ', 'United states', 'u.s.a.',
       'USAUSAUSA', 'america', 'france', 'United States of America ',
       'U.S.A.', 'Mexico', 'Canada ', 'united states of america',
       'US of A', 'UK', 'Unites States', 'The United States',
       'North Carolina ', 'Netherlands', 'Europe', 'Earth', 'U S', 'u.s.',
       'Costa Rica', 'The United States of America', 'cascadia',
       'insanity lately', 'Greece', 'USA? Hard to tell anymore..',
       "'merica", 'Pittsburgh', 'United State', 'France', 'U.S.', 'A',
       'Can', 'Canae', 'New York', 'Trumpistan', 'Ireland',
       'United Sates', 'Korea', 'Australia', 'California', 'Japan', 'USa',
       'I pretend to be from Canada, but I am really from the United States.',
       'Usa ', 'Iceland', 'Canada`'

There are too many 129 countries with varying names. I will just try to correct country name for US.

First changing all contry names to uppercase.

In [12]:
df['Q4: COUNTRY'] = df['Q4: COUNTRY'].str.upper()

In [13]:
df['Q4: COUNTRY'].unique()

array(['USA ', 'US', 'USA', nan, 'CANADA', 'MURICA', 'UK',
       'UNITED STATES', 'UNITED STAES', 'UNITED KINGDOM',
       'UNITED STATES OF AMERICA', 'UAE', 'UNITED STATES ', 'CANADA ',
       'U.S.A.', 'USAUSAUSA', 'AMERICA', 'FRANCE',
       'UNITED STATES OF AMERICA ', 'MEXICO', 'US OF A', 'UNITES STATES',
       'THE UNITED STATES', 'NORTH CAROLINA ', 'NETHERLANDS', 'EUROPE',
       'EARTH', 'U S', 'U.S.', 'COSTA RICA',
       'THE UNITED STATES OF AMERICA', 'CASCADIA', 'INSANITY LATELY',
       'GREECE', 'USA? HARD TO TELL ANYMORE..', "'MERICA", 'PITTSBURGH',
       'UNITED STATE', 'A', 'CAN', 'CANAE', 'NEW YORK', 'TRUMPISTAN',
       'IRELAND', 'UNITED SATES', 'KOREA', 'AUSTRALIA', 'CALIFORNIA',
       'JAPAN',
       'I PRETEND TO BE FROM CANADA, BUT I AM REALLY FROM THE UNITED STATES.',
       'ICELAND', 'CANADA`', 'SCOTLAND', 'DENMARK', 'FRANCE ',
       'SWITZERLAND', 'AHEM....AMERCA', 'SOUTH KOREA', 'NEW JERSEY',
       'UNITED STATED', 'GERMANY', 'UNITED STATSS', 'ENDLAND

In [14]:
df['Q4: COUNTRY'].unique().size

79

Now will correct name for country of USA

In [15]:
usa_names_map = {
    'USA': 'USA',
    'USA ': 'USA',
    'US': 'USA',
    'UNITED STATES': 'USA',
    'U.S.': 'USA',
    'UNITED STATES OF AMERICA': 'USA',
    'US OF A': 'USA',
    'U.S.A.': 'USA',
    'USAUSAUSA': 'USA',
    'THE UNITED STATES':'USA',
    'UNIED STATES': 'USA',
    'USA! USA! USA!':'USA',
    'USAA': 'USA',
    'AMERICA': 'USA',
    'UNHINGED STATES': 'USA',
    'THE UNITED STATES OF AMERICA': 'USA', 
    'UNITE STATES': 'USA',
    'UNITED SATES': 'USA',
    'UNITES STATES': 'USA',
    'UNITED STSTES': 'USA',
    'UNITED STATES ': 'USA',
    'UNITED STATES OF AMERICA ': 'USA',
    'U S': 'USA',
    'UNITED STATED': 'USA',
    'USA USA USA!!!!': 'USA'
}

df['Q4: COUNTRY'] = df['Q4: COUNTRY'].map(usa_names_map).fillna(df['Q4: COUNTRY'])

In [16]:
df['Q4: COUNTRY'].unique()

array(['USA', nan, 'CANADA', 'MURICA', 'UK', 'UNITED STAES',
       'UNITED KINGDOM', 'UAE', 'CANADA ', 'FRANCE', 'MEXICO',
       'NORTH CAROLINA ', 'NETHERLANDS', 'EUROPE', 'EARTH', 'COSTA RICA',
       'CASCADIA', 'INSANITY LATELY', 'GREECE',
       'USA? HARD TO TELL ANYMORE..', "'MERICA", 'PITTSBURGH',
       'UNITED STATE', 'A', 'CAN', 'CANAE', 'NEW YORK', 'TRUMPISTAN',
       'IRELAND', 'KOREA', 'AUSTRALIA', 'CALIFORNIA', 'JAPAN',
       'I PRETEND TO BE FROM CANADA, BUT I AM REALLY FROM THE UNITED STATES.',
       'ICELAND', 'CANADA`', 'SCOTLAND', 'DENMARK', 'FRANCE ',
       'SWITZERLAND', 'AHEM....AMERCA', 'SOUTH KOREA', 'NEW JERSEY',
       'GERMANY', 'UNITED STATSS', 'ENDLAND', 'ATLANTIS', 'MURRIKA',
       'ALASKA', 'N. AMERICA', 'USSA', 'TAIWAN', 'CHINA', 'SPAIN',
       'U.S. ', 'NARNIA', 'U S A', 'UNITED STATEA',
       'SUBSCRIBE TO DM4UZ3 ON YOUTUBE', "I DON'T KNOW ANYMORE",
       'FEAR AND LOATHING'], dtype=object)

In [17]:
df['Q4: COUNTRY'].unique().size

61

Oops , it looks like too many corrections required for fixing country name. I won't fix it anymore as its taking lot of time. I am not sure if there is any better way to do this.

On side note, I think survey form should have dropdown rather than open text field for country. This can help with standardization and cleaner data collection.

## Chapter 8

I will perform below 2 tasks 
* Create hierarchical index
* Reshape

Candy names asked were part of Question 6 in survey. All Candy's surveyed are marked with Columns name starting with "Q6 |". Will extract column names and rename with new name removing "Q6 |".

In [18]:
candy_names = {x:x.split('Q6 | ')[1] for x in df.columns if x.startswith('Q6 | ')}

In [19]:
#candy_names = [ {x:x.split('Q6 | ')[1]} for x in df.columns if x.startswith('Q6 | ')] 
#candy_names
df.rename(columns=candy_names,inplace=True)
df.reset_index(inplace=True)
#df.set_index(list(candy_names.values()),inplace=True)
#df.head()

Dropping records where any of the Gender and Age field have missing values.

In [20]:
df.dropna(subset=['Q2: GENDER','Q3: AGE'],inplace=True)
df.shape

(1743, 113)

### Created Hierachial Data using columns Gender and Age

In [21]:
df.set_index(['Q2: GENDER','Q3: AGE'],inplace=True)
df.head()

Internal ID  ... Click Coordinates (x, y)
Q2: GENDER Q3: AGE               ...                         
Male       44          90272821  ...                 (84, 25)
           40          90272840  ...                 (75, 23)
           23          90272841  ...                 (70, 10)
           33          90272854  ...                  (55, 5)
           40          90272858  ...                 (76, 24)

[5 rows x 111 columns]

In [22]:
new_index = pd.Index(list(candy_names.values()),name='candy')
new_df = df.reindex(columns=new_index)
new_df.head()

candy              100 Grand Bar  ... York Peppermint Patties
Q2: GENDER Q3: AGE                ...                        
Male       44                MEH  ...                 DESPAIR
           40                MEH  ...                 DESPAIR
           23                JOY  ...                     JOY
           33                JOY  ...                 DESPAIR
           40                JOY  ...                 DESPAIR

[5 rows x 103 columns]

### Reshaping data using stack() method

In [23]:
ldata = new_df.stack().reset_index().rename(columns={0: 'candy_liking'})
ldata[:10]

,Q2: GENDER,Q3: AGE,candy,candy_liking
0,Male,44,100 Grand Bar,MEH
1,Male,44,Anonymous brown globs that come in black and o...,DESPAIR
2,Male,44,Any full-sized candy bar,JOY
3,Male,44,Black Jacks,MEH
4,Male,44,Bonkers (the candy),DESPAIR
5,Male,44,Bonkers (the board game),DESPAIR
6,Male,44,Bottle Caps,DESPAIR
7,Male,44,Box'o'Raisins,DESPAIR
8,Male,44,Broken glow stick,DESPAIR
9,Male,44,Butterfinger,DESPAIR


## Chapter 10

I will perform following 2 tasks

* Grouping with Dicts/Series
* Cross Tabs

I will use the dataframe created earlier i.e. df

In [24]:
df.reset_index(inplace=True)
df.head()

,Q2: GENDER,Q3: AGE,Internal ID,Q1: GOING OUT?,Q4: COUNTRY,"Q5: STATE, PROVINCE, COUNTY, ETC",100 Grand Bar,Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),Any full-sized candy bar,Black Jacks,Bonkers (the candy),Bonkers (the board game),Bottle Caps,Box'o'Raisins,Broken glow stick,Butterfinger,Cadbury Creme Eggs,Candy Corn,Candy that is clearly just the stuff given out for free at restaurants,Caramellos,"Cash, or other forms of legal tender",Chardonnay,Chick-o-Sticks (we donÕt know what that is),Chiclets,Coffee Crisp,Creepy Religious comics/Chick Tracts,Dental paraphenalia,Dots,Dove Bars,Fuzzy Peaches,Generic Brand Acetaminophen,Glow sticks,Goo Goo Clusters,Good N' Plenty,Gum from baseball cards,Gummy Bears straight up,Hard Candy,Healthy Fruit,Heath Bar,Hershey's Dark Chocolate,...,Necco Wafers,Nerds,Nestle Crunch,Now'n'Laters,Peeps,Pencils,Pixy Stix,Real Housewives of Orange County Season 9 Blue-Ray,ReeseÕs Peanut Butter Cups,Reese's Pieces,Reggie Jackson Bar,Rolos,Sandwich-sized bags filled with BooBerry Crunch,Skittles,Smarties (American),Smarties (Commonwealth),Snickers,Sourpatch Kids (i.e. abominations of nature),Spotted Dick,Starburst,Sweet Tarts,Swedish Fish,Sweetums (a friend to diabetes),Take 5,Tic Tacs,Those odd marshmallow circus peanut things,Three Musketeers,Tolberone something or other,Trail Mix,Twix,"Vials of pure high fructose corn syrup, for main-lining into your vein",Vicodin,Whatchamacallit Bars,White Bread,Whole Wheat anything,York Peppermint Patties,Q10: DRESS,Q11: DAY,Q12: MEDIA [Science],"Click Coordinates (x, y)"
0,Male,44,90272821,No,USA,NM,MEH,DESPAIR,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,MEH,DESPAIR,MEH,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,DESPAIR,MEH,MEH,DESPAIR,MEH,JOY,...,DESPAIR,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,JOY,JOY,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,JOY,JOY,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,White and gold,Sunday,1.0,"(84, 25)"
1,Male,40,90272840,No,USA,or,MEH,DESPAIR,JOY,MEH,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,MEH,DESPAIR,DESPAIR,MEH,JOY,MEH,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,JOY,JOY,MEH,MEH,JOY,JOY,MEH,DESPAIR,JOY,MEH,MEH,DESPAIR,MEH,...,DESPAIR,MEH,MEH,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,JOY,MEH,MEH,MEH,DESPAIR,DESPAIR,DESPAIR,MEH,JOY,MEH,DESPAIR,MEH,MEH,JOY,MEH,JOY,MEH,DESPAIR,DESPAIR,JOY,MEH,JOY,DESPAIR,JOY,JOY,DESPAIR,DESPAIR,DESPAIR,White and gold,Sunday,1.0,"(75, 23)"
2,Male,23,90272841,No,USA,exton pa,JOY,DESPAIR,JOY,DESPAIR,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,MEH,DESPAIR,DESPAIR,JOY,MEH,JOY,DESPAIR,DESPAIR,MEH,DESPAIR,DESPAIR,MEH,JOY,DESPAIR,DESPAIR,DESPAIR,MEH,MEH,DESPAIR,JOY,MEH,DESPAIR,DESPAIR,JOY,...,DESPAIR,JOY,JOY,JOY,MEH,DESPAIR,JOY,DESPAIR,JOY,JOY,MEH,JOY,NaN,JOY,JOY,DESPAIR,NaN,MEH,DESPAIR,JOY,JOY,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,JOY,JOY,DESPAIR,JOY,MEH,JOY,JOY,DESPAIR,DESPAIR,JOY,White and gold,Friday,1.0,"(70, 10)"
3,Male,33,90272854,No,CANADA,ontario,JOY,DESPAIR,JOY,DESPAIR,DESPAIR,MEH,JOY,MEH,JOY,JOY,JOY,JOY,DESPAIR,JOY,JOY,JOY,DESPAIR,MEH,MEH,DESPAIR,DESPAIR,MEH,MEH,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,JOY,...,MEH,JOY,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,JOY,JOY,NaN,JOY,DESPAIR,JOY,MEH,JOY,MEH,JOY,DESPAIR,MEH,JOY,JOY,MEH,MEH,DESPAIR,DESPAIR,JOY,MEH,DESPAIR,JOY,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,Blue and black,Friday,1.0,"(55, 5)"
4,Male,40,90272858,No,CANADA,Ontario,JOY,DESPAIR,JOY,MEH,MEH,MEH,MEH,DESPAIR,DESPAIR,JOY,MEH,DESPAIR,DESPAIR,MEH,MEH,JOY,MEH,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,JOY,MEH,DESPAIR,MEH,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,JOY,...,DESPAIR,MEH,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,JOY,JOY,DESPAIR,JOY,DESPAIR,DESPAIR,MEH,JOY,JOY,JOY,DESPAIR,DESPAIR,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,JOY,DESPAIR,JOY,MEH,DESPAIR,MEH,DESPAIR,DESPAIR,DESPAIR,Blue and black,Su

In [25]:
df.shape

(1743, 113)

In [26]:
# dropping records where either country or State/Province/City field have missing data
df.dropna(subset=['Q4: COUNTRY','Q5: STATE, PROVINCE, COUNTY, ETC'],inplace=True)
df.shape

(1723, 113)

Now will group by Country and City. 

For that purpose lets create a mapping of columns

In [27]:
mapping = {
    'Q4: COUNTRY': 'COUNTRY',
    'Q5: STATE, PROVINCE, COUNTY, ETC': 'LOCATION'
}

by_column = df.groupby(mapping, axis=1)
by_column

In [28]:
by_column.describe()


,,count,unique,top,freq
COUNTRY,Q4: COUNTRY,1723,56,USA,1490
LOCATION,"Q5: STATE, PROVINCE, COUNTY, ETC",1723,423,California,107


Key Highlights
* Total Survey results - 1723
* Number of unique countries from where survey resulst are collected - 56
* Highest participation was for country USA and total respondents were 1490
* Number of unique locations i.e. State/City from survey results are collected - 423
* Highest participation in survey was from state of California i.e. 107

### CrossTab

Now I would like to find count of each candy marked as JOY , MEH and DISPAIR.
I will use CrossTab function.

I also want to find most liked Candy.

For this analysis , I will use dataframe created in Chapter 8 Exercise i.e. "ldata"

In [29]:
ldata[:10]

,Q2: GENDER,Q3: AGE,candy,candy_liking
0,Male,44,100 Grand Bar,MEH
1,Male,44,Anonymous brown globs that come in black and o...,DESPAIR
2,Male,44,Any full-sized candy bar,JOY
3,Male,44,Black Jacks,MEH
4,Male,44,Bonkers (the candy),DESPAIR
5,Male,44,Bonkers (the board game),DESPAIR
6,Male,44,Bottle Caps,DESPAIR
7,Male,44,Box'o'Raisins,DESPAIR
8,Male,44,Broken glow stick,DESPAIR
9,Male,44,Butterfinger,DESPAIR


Now Running crosstab function over candy and candy_liking column.

In [30]:
pd.crosstab([ldata['candy']],ldata.candy_liking,margins=True)

candy_liking,DESPAIR,JOY,MEH,All
candy,,,,
100 Grand Bar,84,852,732,1668
Abstained from M&M'ing.,681,217,592,1490
Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),1055,174,452,1681
Any full-sized candy bar,16,1517,201,1734
Black Jacks,778,87,601,1466
...,...,...,...,...
Whatchamacallit Bars,278,831,495,1604
White Bread,1417,43,202,1662
Whole Wheat anything,1257,114,303,1674


Lets find most liked candy.

In [31]:
crosstab_df = pd.crosstab([ldata['candy']],ldata.candy_liking)
print(crosstab_df[crosstab_df['JOY'] == crosstab_df['JOY'].max()])

candy_liking              DESPAIR   JOY  MEH
candy                                       
Any full-sized candy bar       16  1517  201


It looks like participants were more focused on size of candy rather than favour of the candy. "Any full size candy bar" was mosted liked candy in the survey.

## Chapter 11

I will perform following tasks
* Convert between string and date time
* Convert timestamps to periods and back


The dataset file copied from site [So Much Data Candy, Seriously (Ng, 2017)](https://www.scq.ubc.ca/wp-content/uploads/2017/10/candyhierarchy2017.csv)  does not contain any date field. So I will be using older file i.e. 2016 survey file which I downloaded from Bellevue University site [Week 7 & 8 Datasets](http://content.bellevue.edu/cst/dsc/540/id/data-files/12-week/weeks-7-&-8.zip). The filename is BOING-BOING-CANDY-HIERARCHY-2016-SURVEY-Responses.xlsx

In [32]:
na_values = [""," ",None,"Missing","NA"]
df_2016= pd.read_excel("BOING-BOING-CANDY-HIERARCHY-2016-SURVEY-Responses.xlsx",na_values=na_values)
df_2016.head()

,Timestamp,Are you going actually going trick or treating yourself?,Your gender:,How old are you?,Which country do you live in?,"Which state, province, county do you live in?",[100 Grand Bar],[Anonymous brown globs that come in black and orange wrappers],[Any full-sized candy bar],[Black Jacks],[Bonkers (the candy)],[Bonkers (the board game)],[Bottle Caps],[Box'o'Raisins],[Broken glow stick],[Butterfinger],[Cadbury Creme Eggs],[Candy Corn],[Candy that is clearly just the stuff given out for free at restaurants],[Caramellos],"[Cash, or other forms of legal tender]",[Chardonnay],[Chick-o-Sticks (we don’t know what that is)],[Chiclets],[Coffee Crisp],[Creepy Religious comics/Chick Tracts],[Dental paraphenalia],[Dots],[Dove Bars],[Fuzzy Peaches],[Generic Brand Acetaminophen],[Glow sticks],[Goo Goo Clusters],[Good N' Plenty],[Gum from baseball cards],[Gummy Bears straight up],[Hard Candy],[Healthy Fruit],[Heath Bar],[Hershey's Dark Chocolate],...,[Rolos],[Skittles],[Smarties (American)],[Smarties (Commonwealth)],[Snickers],[Sourpatch Kids (i.e. abominations of nature)],[Spotted Dick],[Starburst],[Sweet Tarts],[Swedish Fish],[Sweetums (a friend to diabetes)],[Tic Tacs],[Those odd marshmallow circus peanut things],[Three Musketeers],[Tolberone something or other],[Trail Mix],[Twix],"[Vials of pure high fructose corn syrup, for main-lining into your vein]",[Vicodin],[Whatchamacallit Bars],[White Bread],[Whole Wheat anything],[York Peppermint Patties],Please list any items not included above that give you JOY.,Please list any items not included above that give you DESPAIR.,"Please leave any witty, snarky or thoughtful remarks or comments regarding your choices.",Guess the number of mints in my hand.,Betty or Veronica?,"""That dress* that went viral a few years back - when I first saw it, it was ________""",What is your favourite font?,Please estimate the degree(s) of separation you have from the following celebrities [JK Rowling],Please estimate the degree(s) of separation you have from the following celebrities [JJ Abrams],Please estimate the degree(s) of separation you have from the following celebrities [Beyoncé],Please estimate the degree(s) of separation you have from the following celebrities [Bieber],Please estimate the degree(s) of separation you have from the following celebrities [Kevin Bacon],Please estimate the degree(s) of separation you have from the following celebrities [Francis Bacon (1561 - 1626)],"Which day do you prefer, Friday or Sunday?","Do you eat apples the correct way, East to West (side to side) or do you eat them like a freak of nature, South to North (bottom to top)?","When you see the above image of the 4 different websites, which one would you most likely check out (please be honest).",[York Peppermint Patties] Ignore
0,2016-10-24 05:09:23.033,No,Male,22,Canada,Ontario,JOY,DESPAIR,JOY,MEH,MEH,MEH,MEH,DESPAIR,DESPAIR,JOY,JOY,DESPAIR,DESPAIR,JOY,JOY,MEH,MEH,MEH,JOY,JOY,DESPAIR,DESPAIR,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,JOY,DESPAIR,JOY,JOY,DESPAIR,DESPAIR,JOY,...,JOY,JOY,DESPAIR,JOY,JOY,JOY,DESPAIR,JOY,JOY,JOY,DESPAIR,DESPAIR,DESPAIR,JOY,JOY,DESPAIR,JOY,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,JOY,NaN,NaN,NaN,200000,Betty,White and gold,impact,3 or higher,2,3 or higher,3 or higher,3 or higher,3 or higher,Friday,South to North,Science: Latest News and Headlines,NaN
1,2016-10-24 05:09:54.798,No,Male,45,usa,il,MEH,MEH,JOY,JOY,DESPAIR,MEH,JOY,DESPAIR,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,JOY,JOY,JOY,JOY,DESPAIR,MEH,DESPAIR,DESPAIR,JOY,MEH,DESPAIR,MEH,MEH,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,JOY,JOY,...,JOY,JOY,DESPAIR,DESPAIR,JOY,DESPAIR,MEH,JOY,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,JOY,JOY,MEH,JOY,DESPAIR,JOY,JOY,DESPAIR,DESPAIR,JOY,NaN,NaN,NaN,17,Betty,White and gold,Calibri,3 or higher,3 or higher,3 or higher,3 or higher,3 or higher,3 or higher,Friday,East to West,Science: Latest News and Headlines,NaN
2,2016-10-24 05:13:06.734,No,Female,48,US,Colorado,JOY,DESPAIR,JOY,MEH,MEH,JOY,JOY,DESPAIR,DESPAIR,JOY,MEH,ME

Checking Data type of Timestamp column.

In [33]:
dataTypeObj = df_2016.dtypes['Timestamp']
print(dataTypeObj)

datetime64[ns]


Dataset is loaded in panda dataframe with Timestamp column as type datetime64.

Lets me add day of the week field as extra column in dataframe. I will extract that value from timestamp field.

In [34]:
day_of_week = df_2016['Timestamp'].apply(lambda x: x.strftime('%A'))
# Adding day_of_week as 2nd column in dataframe
df_2016.insert(loc=1,column='Day_of_Week',value=day_of_week)
df_2016.sample(5)

,Timestamp,Day_of_Week,Are you going actually going trick or treating yourself?,Your gender:,How old are you?,Which country do you live in?,"Which state, province, county do you live in?",[100 Grand Bar],[Anonymous brown globs that come in black and orange wrappers],[Any full-sized candy bar],[Black Jacks],[Bonkers (the candy)],[Bonkers (the board game)],[Bottle Caps],[Box'o'Raisins],[Broken glow stick],[Butterfinger],[Cadbury Creme Eggs],[Candy Corn],[Candy that is clearly just the stuff given out for free at restaurants],[Caramellos],"[Cash, or other forms of legal tender]",[Chardonnay],[Chick-o-Sticks (we don’t know what that is)],[Chiclets],[Coffee Crisp],[Creepy Religious comics/Chick Tracts],[Dental paraphenalia],[Dots],[Dove Bars],[Fuzzy Peaches],[Generic Brand Acetaminophen],[Glow sticks],[Goo Goo Clusters],[Good N' Plenty],[Gum from baseball cards],[Gummy Bears straight up],[Hard Candy],[Healthy Fruit],[Heath Bar],...,[Rolos],[Skittles],[Smarties (American)],[Smarties (Commonwealth)],[Snickers],[Sourpatch Kids (i.e. abominations of nature)],[Spotted Dick],[Starburst],[Sweet Tarts],[Swedish Fish],[Sweetums (a friend to diabetes)],[Tic Tacs],[Those odd marshmallow circus peanut things],[Three Musketeers],[Tolberone something or other],[Trail Mix],[Twix],"[Vials of pure high fructose corn syrup, for main-lining into your vein]",[Vicodin],[Whatchamacallit Bars],[White Bread],[Whole Wheat anything],[York Peppermint Patties],Please list any items not included above that give you JOY.,Please list any items not included above that give you DESPAIR.,"Please leave any witty, snarky or thoughtful remarks or comments regarding your choices.",Guess the number of mints in my hand.,Betty or Veronica?,"""That dress* that went viral a few years back - when I first saw it, it was ________""",What is your favourite font?,Please estimate the degree(s) of separation you have from the following celebrities [JK Rowling],Please estimate the degree(s) of separation you have from the following celebrities [JJ Abrams],Please estimate the degree(s) of separation you have from the following celebrities [Beyoncé],Please estimate the degree(s) of separation you have from the following celebrities [Bieber],Please estimate the degree(s) of separation you have from the following celebrities [Kevin Bacon],Please estimate the degree(s) of separation you have from the following celebrities [Francis Bacon (1561 - 1626)],"Which day do you prefer, Friday or Sunday?","Do you eat apples the correct way, East to West (side to side) or do you eat them like a freak of nature, South to North (bottom to top)?","When you see the above image of the 4 different websites, which one would you most likely check out (please be honest).",[York Peppermint Patties] Ignore
623,2016-10-24 15:15:47.759,Monday,No,Male,45,usa,colo,MEH,DESPAIR,JOY,MEH,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,JOY,JOY,DESPAIR,MEH,JOY,MEH,MEH,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,MEH,DESPAIR,JOY,MEH,JOY,DESPAIR,MEH,DESPAIR,DESPAIR,JOY,...,DESPAIR,MEH,JOY,DESPAIR,MEH,DESPAIR,DESPAIR,DESPAIR,MEH,JOY,DESPAIR,MEH,JOY,JOY,MEH,MEH,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,JOY,NaN,cabbage,NaN,5,Betty,White and gold,palatino,2,3 or higher,3 or higher,3 or higher,2,2,Sunday,East to West,Science: Latest News and Headlines,NaN
309,2016-10-24 08:50:08.365,Monday,Yes,Male,43,USA,OR,JOY,DESPAIR,JOY,DESPAIR,MEH,JOY,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,JOY,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,MEH,MEH,DESPAIR,JOY,MEH,JOY,DESPAIR,DESPAIR,JOY,DESPAIR,MEH,DESPAIR,...,MEH,JOY,JOY,DESPAIR,JOY,JOY,JOY,JOY,JOY,MEH,DESPAIR,MEH,DESPAIR,JOY,MEH,MEH,JOY,DESPAIR,JOY,JOY,DESPAIR,DESPAIR,JOY,Spree,NaN,NaN,NaN,Betty,Blue and black,Trump,3 or higher,3 or higher,3 or higher,3 or higher,3 or higher,3 or higher,Friday,East to West,Daily Dish,NaN
648,2016-10-24 16:28:17.087,Monday,No,Female,36,United States,CA,JOY,DESPAIR,JOY,DESPAIR,NaN,NaN,JOY,DESPAIR,DESPAIR,DESPAIR,JOY,DESPAIR,DESPAIR,NaN,JOY,NaN,NaN,MEH,ME

In [37]:
print(f" Data collected is between date range {df_2016['Timestamp'].min()} - {df_2016['Timestamp'].max()}")

 Data collected is between date range 2016-10-24 05:09:23.033000 - 2016-10-30 17:06:45.660000


SO data is collected during 7 days for 10/24 to 10/30 in year 2016.

### Convert timestamps to periods and back

I will convert timestamp into periods i.e. Morning, Afternoo, Evening and Night.
Then I will find how many survey results collected in those periods.

In [39]:
session=pd.cut(df_2016.Timestamp.dt.hour,
               [0,6,12,18,23],
               labels=['Night','Morning','Afternoon','Evening'],
               include_lowest=True)
session.value_counts()

Morning      586
Afternoon    362
Night        218
Evening       93
Name: Timestamp, dtype: int64